## Homework: Agents

In this homework, we will learn more about function calling,
and we will also explore MCP - model-context protocol. 


## Preparation

First, we'll define a function that we will use when building
our agent. 

It will generate fake weather data:

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

We want to use it as a tool for our agent, so we need to 
describe it 

How should the description for this function look like? Fill in missing parts

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get randomly generated weather data from a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Get a weather data from given city"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

In [3]:
import chat_assistant

In [4]:
from openai import OpenAI
client = OpenAI()

In [5]:
tools = chat_assistant.Tools()

In [6]:
tools.add_tool(get_weather, get_weather_tool)
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get randomly generated weather data from a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Get a weather data from given city'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [7]:
developer_prompt = """
You are an answering machine
You're given a question by a human and your task is to answer it
"""
chat_interface = chat_assistant.ChatInterface()


In [8]:
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [9]:
chat.run()


Chat ended.


# Q2. Adding another tool

Let's add another tool - a function that can add weather data
to our database:

```python
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
```

Now let's write a description for it.

What did you write?

Optionally, you can test it after adding this function.

In [10]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [11]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "add weather data entry into known_weather_data",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "set the city name "
            },
            "temp": {
                "type": "number",
                "description": "Set the temperature"
            }
        },
        "required": ["city","temp"],
        "additionalProperties": False
    }
}

In [12]:

tools.add_tool(set_weather,set_weather_tool)
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get randomly generated weather data from a city',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Get a weather data from given city'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'add weather data entry into known_weather_data',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'set the city name '},
    'temp': {'type': 'number', 'description': 'Set the temperature'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [13]:
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [14]:
chat.run()

Chat ended.


In [15]:
print(known_weather_data)

{'berlin': 20.0, 'melbourne': 6.5}


## MCP

MCP stands for Model-Context Protocol. It allows LLMs communicate
with different tools (like Qdrant). It's function calling, but 
one step further:

* A tool can export a list of functions it has
* When we include the tool to our Agent, we just need to include the link to the MCP server

## Q3. Install FastMCP

Let's install a library for MCP - [FastMCP](https://github.com/jlowin/fastmcp):

```bash
pip install fastmcp
```

What's the version of FastMCP you installed?

In [16]:
!uv pip install fastmcp

Using Python 3.13.5 environment at: /Users/robertusprimusanto/Documents/code/dataTalksClub/llmzoomcamp-202506/homework-llmzoomcamp-2025/.venv
Audited 1 package in 16ms


In [17]:
!fastmcp version

FastMCP version:                                                          2.10.5
MCP version:                                                              1.11.0
Python version:                                                           3.13.5
Platform:                                      macOS-15.5-arm64-arm-64bit-Mach-O
FastMCP root path: /Users/robertusprimusanto/Documents/code/dataTalksClub/llmzo…


Q4. Simple MCP Server

In [ ]:
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city":"berlin"}}}


/opt/homebrew/Cellar/python@3.13/3.13.5/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=43895) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()




╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                             │
│    📦 Transport:       STDIO                                               │
│                                                  

In [29]:
from fastmcp import Client
import weather_server

async def main():
    print("\n--- Testing Server Locally ---")
    mcp_client = Client(weather_server.mcp)
    async with mcp_client:
        tools = await mcp_client.list_tools()
        print(f"Available tools: {tools}")


tool_list = await main()


--- Testing Server Locally ---
Available tools: [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title':

In [33]:
print(await main())


--- Testing Server Locally ---
Available tools: [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title':

In [34]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully


In [35]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '20.0'}],
 'structuredContent': {'result': 20.0},
 'isError': False}

In [36]:
import json

class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None
    
    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

In [37]:
our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'jakarta'}


Calling tool 'set_weather' with arguments: {'city': 'melbourne', 'temp': 5}


Chat ended.
